In [2]:
import os
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import numpy as np
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.nn.conv import GCNConv
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, DataStructs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
from e3fp.fingerprint.generate import fp, fprints_dict_from_mol
from e3fp.conformer.generate import generate_conformers
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
IPythonConsole.ipython_useSVG=True
%matplotlib inline

In [3]:
# 判断是用GPU或CPU计算
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 读取SDF文件
sdf_file = './dataset/BindingDB_PubChem_3D.sdf'
# Pandas Dataframe的列名
df = pd.DataFrame(columns=['smiles', 'mol_weight', 'IC50 (nM)'])
# 读取的分子数量
chunk_size = 10000
mols = []

In [4]:
suppl = Chem.SDMolSupplier(sdf_file)
 # 用RDKit中的SDMolSupplier读取分子
for i,mol in enumerate(suppl):
    if i > 0 and i % chunk_size == 0:
        break
    else:
        if mol is not None:
            mol = Chem.AddHs(mol)
            mols.append(mol)
            #其他非标准的属性
            # propNames = list(mol.GetPropNames())
            # print(propNames)
            # 提取分子信息
            mol_ID = mol.GetProp('_Name')
            smiles = Chem.MolToSmiles(mol)
            mol_weight = Chem.rdMolDescriptors.CalcExactMolWt(mol)
            IC50 = mol.GetProp('IC50 (nM)')
          
            # 添加数据到DataFrame
            df = df.append({'smiles': smiles, 'mol_weight': mol_weight,
                            'IC50 (nM)': IC50}, ignore_index=True)
 # 打印结果
df

,smiles,mol_weight,IC50 (nM)
0,[H]N=c1c(C(=O)N([H])C([H])([H])c2oc([H])c([H])...,439.164440,15300
1,[H]c1c(OC([H])([H])[H])c([H])c2c([H])c(C#N)c(N...,253.121512,100000
2,[H]OC(=O)C1([H])N([H])c2c(OC([H])([H])[H])c([H...,245.105193,15400
3,[H]c1c([H])c([H])c2c(c1[H])-c1c([H])c(C([H])([...,304.168797,100000
4,[H]N=c1c(S(=O)(=O)c2c([H])c([H])c([H])c([H])c2...,432.089226,57500
...,...,...,...
9787,[H]Oc1c([H])c([H])c(N2C(=O)C([H])(C([H])([H])C...,324.147393,
9788,[H]c1c(OC([H])([H])[H])c([H])c2c(N([H])C([H])(...,399.207740,
9789,[H]C1=C2C([H])([H])[C@@]([H])(SC(=O)C([H])([H]...,416.202131,
9790,[H]c1c([H])c([H])c(-c2c3c([H])c(Br)c(=O)c([H])...,686.837763,


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9792 entries, 0 to 9791
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   smiles      9792 non-null   object 
 1   mol_weight  9792 non-null   float64
 2   IC50 (nM)   9792 non-null   object 
dtypes: float64(1), object(2)
memory usage: 229.6+ KB


In [5]:
# 数据清洗
df['IC50 (nM)'] = pd.to_numeric(df['IC50 (nM)'], errors='coerce')
# 删除空值
df = df.dropna()
df.info()
df = df.reset_index(drop=True)
# 围缩小数据范围并使其更符合正态分布，利于后续归一化
df['IC50 (nM)'] = np.log10(df['IC50 (nM)'])
df['mol_weight'] = np.log10(df['mol_weight'])
# 归一化
# df['IC50 (nM)'] = (df['IC50 (nM)'] - df['IC50 (nM)'].min()) / (df['IC50 (nM)'].max() - df['IC50 (nM)'].min())
print(df['IC50 (nM)'].min(), df['IC50 (nM)'].max())
df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5036 entries, 0 to 9791
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   smiles      5036 non-null   object 
 1   mol_weight  5036 non-null   float64
 2   IC50 (nM)   5036 non-null   float64
dtypes: float64(2), object(1)
memory usage: 157.4+ KB
0.8175653695597808 5.670493552992655


,smiles,mol_weight,IC50 (nM)
0,[H]N=c1c(C(=O)N([H])C([H])([H])c2oc([H])c([H])...,2.642627,4.184691
1,[H]c1c(OC([H])([H])[H])c([H])c2c([H])c(C#N)c(N...,2.403329,5.000000
2,[H]OC(=O)C1([H])N([H])c2c(OC([H])([H])[H])c([H...,2.389353,4.187521
3,[H]c1c([H])c([H])c2c(c1[H])-c1c([H])c(C([H])([...,2.483115,5.000000
4,[H]N=c1c(S(=O)(=O)c2c([H])c([H])c([H])c([H])c2...,2.635573,4.759668
...,...,...,...
5031,[H]c1c([H])c2c(c([H])c1OC([H])([H])C([H])([H])...,2.613056,4.480725
5032,[H]c1c(N([H])[H])c([H])c2c(c1[H])c([H])c1c([H]...,2.350477,3.756636
5033,[H]c1c(N([H])[H])c([H])c2nc3c([H])c(N([H])[H])...,2.320344,3.948413
5034,[H]c1c(OC([H])([H])[H])c([H])c2c(N([H])C([H])(...,2.601199,4.298416


In [1]:
df3d = {"MOLI":[], "MOLJ":[], "E3FPTC":[], "pairidx":[]}
fpdicts = [ fprints_dict_from_mol( mol ) for mol in mols ]
# get e3fp fingerprint
# if molecule has multiple conformers the function will generate multiple fingerprints.
fps = [ fp[5][0] for fp in fpdicts]
# convert to rdkit fp from e3fp fingerprint
binfp = [ fp.fold().to_rdkit() for fp in fps ]
# calculate tanimoto similarity
for i in range( len(binfp) ):
    for j in range( i ):
        e3fpTC = DataStructs.TanimotoSimilarity( binfp[i], binfp[j] )
        moli = mols[i].GetProp("_Name")
        molj = mols[j].GetProp("_Name")
        df3d["MOLI"].append( moli )
        df3d["MOLJ"].append( molj )
        df3d["E3FPTC"].append( e3fpTC )
        df3d["pairidx"].append( str(i)+"_vs_"+str(j) )
df3d = pd.DataFrame( df3d )
df3d

NameError: name 'mols' is not defined

In [ ]:
# 合并数据,以smiles为键,合并两个DataFrame
data = pd.merge(df, df3d, left_on='smiles', right_on='MOLI', how='left')

In [27]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        smiles = self.data.iloc[idx]['smiles']
        mol_weight = self.data.iloc[idx]['mol_weight']
        ic50 = self.data.iloc[idx]['IC50 (nM)']
        sample = {'smiles': smiles, 'mol_weight': mol_weight, 'ic50': ic50}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [29]:
def normalize(x):
    '''
    归一化
    :param x: 输入数据
    :return: 归一化后的数据
    '''
    return (x - x.min()) / (x.max() - x.min())

In [30]:
class InteractionTransform:
    def __call__(self, sample):
        # 将SMILES字符串转化为分子指纹与邻接矩阵
        adj_matrix = Chem.Get3DDistanceMatrix(sample['smiles'])
        x = torch.tensor(sample[''], dtype=torch.float)
        w = torch.tensor(adj_matrix, dtype=torch.float)
        y = torch.tensor(sample['ic50'], dtype=torch.float)
        # print(x.shape)
        # print(y.shape)
        xy = torch.cat((x, y.unsqueeze(0)))

        return {'h': xy, 'w': w}

In [31]:
class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.gc1 = GCNConv(in_features, hidden_features)
        self.gc2 = GCNConv(hidden_features, out_features)

    def forward(self, h, w):
        h = self.gc1(h, w)
        h = F.relu(h)
        h = F.dropout(h, training=self.training)
        h = self.gc2(h, w)
        return F.log_softmax(h, dim=1)

In [32]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        h = data['h'].to(device)
        w = data['w'].to(device)
        optimizer.zero_grad()
        output = model(h, w)
        loss = nn.TripletMarginLoss(output, w)
        loss.backward()
        optimizer.step()
        # 每10个batch打印一次训练信息
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [33]:
# 测试函数
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data in test_loader:
            h = data['h'].to(device)
            w = data['w'].to(device)
            output = model(h, w)
            # 把损失相加
            test_loss += nn.TripletMarginLoss(output, w, reduction='sum').item() 
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}\n'.format(test_loss))

In [34]:
# 数据集划分
train_data, test_data = train_test_split(df, test_size=0.2)
# 数据加载器
train_dataset = CustomDataset(train_data, transform=InteractionTransform())
test_dataset = CustomDataset(test_data, transform=InteractionTransform())
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# 模型定义
model = GCN(2049, 10, 1).to(device)
# 选择优化器，加入学习率lr，
# 当lr过小->收敛下降过慢，过大->错过局部最优；
# 加入正则化系数weight_decay，防止过拟合
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
epochs = 100

In [35]:
print(len(train_dataset[0]['h']))   # 检查 h 特征的长度
print(len(train_dataset[0]['w']))   # 检查 w 特征的长度

print(model)
print(train_loader.dataset[0]['h'].shape)
print(train_loader.dataset[0]['w'].shape)

AttributeError: module 'rdkit.Chem.Pharm3D' has no attribute 'Generate'

In [ ]:

# 训练模型
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

RuntimeError: stack expects each tensor to be equal size, but got [23, 23] at entry 0 and [29, 29] at entry 1